# Full Dataset Statistical Analysis with RGB Color Classification

This notebook performs comprehensive statistical analysis on the full dataset (401 images) with enhanced color analysis including RGB classification.

## 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import pearsonr, spearmanr, mannwhitneyu, kruskal, chi2_contingency, f_oneway
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print('Libraries imported successfully!')

## 2. Load Full Dataset

In [ ]:
# Load the full dataset
df = pd.read_csv('image_analysis_results_full.csv')

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
df.head()

## 3. Data Preprocessing and RGB Color Classification

In [ ]:
# Extract RGB values from predominant_color columndef extract_rgb(color_str):    """Extract RGB values from string representation"""    try:        # Remove extra text and extract numbers        color_str = color_str.replace('(', '').replace(')', '').replace('np.int64', '')        parts = [p.strip() for p in color_str.split(',')]        r, g, b = int(parts[0]), int(parts[1]), int(parts[2])        return r, g, b    except Exception:        return None, None, None# Apply extractiondf[['red', 'green', 'blue']] = df['predominant_color'].apply(    lambda x: pd.Series(extract_rgb(x)))# Remove any rows with missing RGB valuesdf = df.dropna(subset=['red', 'green', 'blue'])print(f"\nDataset shape after RGB extraction: {df.shape}")print(f"\nSample RGB values:")df[['predominant_color', 'red', 'green', 'blue']].head(10)

In [ ]:
# Classify colors into RGB broad categoriesdef classify_rgb_category(r, g, b):    """Classify color into broad RGB categories"""    # Thresholds for classification    threshold = 20  # Difference threshold    gray_threshold = 30  # For near-equal RGB values        # Check if achromatic (gray/white/black)    if max(r, g, b) - min(r, g, b) < gray_threshold:        avg = (r + g + b) / 3        if avg < 85:            return 'Achromatic: Black/Dark'        elif avg > 170:            return 'Achromatic: White/Light'        else:            return 'Achromatic: Gray/Neutral'        # Find dominant channel    max_val = max(r, g, b)        # Red dominant    if r == max_val and r > g + threshold and r > b + threshold:        return 'Chromatic: Red Dominant'    # Green dominant    elif g == max_val and g > r + threshold and g > b + threshold:        return 'Chromatic: Green Dominant'    # Blue dominant    elif b == max_val and b > r + threshold and b > g + threshold:        return 'Chromatic: Blue Dominant'    # Mixed colors    elif r > b + threshold and g > b + threshold:        return 'Chromatic: Yellow/Warm (R+G)'    elif r > g + threshold and b > g + threshold:        return 'Chromatic: Magenta/Purple (R+B)'    elif g > r + threshold and b > r + threshold:        return 'Chromatic: Cyan/Cool (G+B)'    else:        return 'Chromatic: Mixed/Balanced'# Apply RGB classificationdf['rgb_category'] = df.apply(    lambda row: classify_rgb_category(row['red'], row['green'], row['blue']),    axis=1)print("\nRGB Category Distribution:")print(df['rgb_category'].value_counts())print(f"\nNumber of unique RGB categories: {df['rgb_category'].nunique()}")

## 4. Basic Descriptive Statistics

In [ ]:
# Handle missing values
print(f"\nMissing values before cleaning:")
print(df[['likes', 'comments', 'engagement_total']].isna().sum())

# Remove rows with missing engagement data
df = df.dropna(subset=['likes', 'comments', 'engagement_total'])

print(f"\nDataset shape after removing missing values: {df.shape}")
print(f"Missing values after cleaning:")
print(df[['likes', 'comments', 'engagement_total']].isna().sum())

In [ ]:
# Summary statistics for numerical columns
numeric_cols = ['width', 'height', 'mean_luminosity', 'median_luminosity', 'std_luminosity',
                'mean_saturation', 'median_saturation', 'std_saturation',
                'predominant_color_percentage', 'likes', 'comments', 'engagement_total',
                'red', 'green', 'blue']

print("\n=== DESCRIPTIVE STATISTICS ===")
print(df[numeric_cols].describe())

In [ ]:
# Distribution of original color categories
print("\n=== ORIGINAL COLOR CATEGORY DISTRIBUTION ===")
print(df['color_category'].value_counts())

print("\n=== COLOR TEMPERATURE DISTRIBUTION ===")
print(df['color_temperature'].value_counts())

## 5. Exploratory Data Analysis - Visualizations

In [ ]:
# Distribution plots for key metrics
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Luminosity distributions
axes[0, 0].hist(df['mean_luminosity'], bins=30, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Mean Luminosity Distribution')
axes[0, 0].set_xlabel('Mean Luminosity')
axes[0, 0].set_ylabel('Frequency')

axes[0, 1].hist(df['std_luminosity'], bins=30, edgecolor='black', alpha=0.7, color='orange')
axes[0, 1].set_title('Std Luminosity Distribution')
axes[0, 1].set_xlabel('Std Luminosity')
axes[0, 1].set_ylabel('Frequency')

# Saturation distributions
axes[0, 2].hist(df['mean_saturation'], bins=30, edgecolor='black', alpha=0.7, color='green')
axes[0, 2].set_title('Mean Saturation Distribution')
axes[0, 2].set_xlabel('Mean Saturation')
axes[0, 2].set_ylabel('Frequency')

# Engagement metrics
axes[1, 0].hist(df['likes'], bins=30, edgecolor='black', alpha=0.7, color='red')
axes[1, 0].set_title('Likes Distribution')
axes[1, 0].set_xlabel('Likes')
axes[1, 0].set_ylabel('Frequency')

axes[1, 1].hist(df['comments'], bins=30, edgecolor='black', alpha=0.7, color='purple')
axes[1, 1].set_title('Comments Distribution')
axes[1, 1].set_xlabel('Comments')
axes[1, 1].set_ylabel('Frequency')

axes[1, 2].hist(df['engagement_total'], bins=30, edgecolor='black', alpha=0.7, color='brown')
axes[1, 2].set_title('Total Engagement Distribution')
axes[1, 2].set_xlabel('Total Engagement')
axes[1, 2].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# RGB distribution
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

axes[0].hist(df['red'], bins=30, edgecolor='black', alpha=0.7, color='red')
axes[0].set_title('Red Channel Distribution')
axes[0].set_xlabel('Red Value (0-255)')
axes[0].set_ylabel('Frequency')

axes[1].hist(df['green'], bins=30, edgecolor='black', alpha=0.7, color='green')
axes[1].set_title('Green Channel Distribution')
axes[1].set_xlabel('Green Value (0-255)')
axes[1].set_ylabel('Frequency')

axes[2].hist(df['blue'], bins=30, edgecolor='black', alpha=0.7, color='blue')
axes[2].set_title('Blue Channel Distribution')
axes[2].set_xlabel('Blue Value (0-255)')
axes[2].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

## 6. Correlation Analysis

In [ ]:
# Select features for correlation analysis
correlation_features = ['mean_luminosity', 'median_luminosity', 'std_luminosity',
                        'mean_saturation', 'median_saturation', 'std_saturation',
                        'predominant_color_percentage', 'red', 'green', 'blue',
                        'likes', 'comments', 'engagement_total']

# Compute Pearson correlation
pearson_corr = df[correlation_features].corr(method='pearson')

# Visualize correlation heatmap
plt.figure(figsize=(14, 12))
sns.heatmap(pearson_corr, annot=True, fmt='.3f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Pearson Correlation Heatmap - Full Dataset', fontsize=16, pad=20)
plt.tight_layout()
plt.show()

In [ ]:
# Spearman correlation (for non-linear relationships)
spearman_corr = df[correlation_features].corr(method='spearman')

plt.figure(figsize=(14, 12))
sns.heatmap(spearman_corr, annot=True, fmt='.3f', cmap='viridis', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Spearman Correlation Heatmap - Full Dataset', fontsize=16, pad=20)
plt.tight_layout()
plt.show()

In [ ]:
# Detailed correlation with engagement metrics
print("\n=== CORRELATIONS WITH LIKES ===")
for feature in correlation_features[:-3]:  # Exclude likes, comments, engagement_total
    pearson_r, pearson_p = pearsonr(df[feature], df['likes'])
    spearman_r, spearman_p = spearmanr(df[feature], df['likes'])
    print(f"{feature:30s} | Pearson: {pearson_r:7.4f} (p={pearson_p:.4f}) | Spearman: {spearman_r:7.4f} (p={spearman_p:.4f})")

print("\n=== CORRELATIONS WITH ENGAGEMENT_TOTAL ===")
for feature in correlation_features[:-3]:
    pearson_r, pearson_p = pearsonr(df[feature], df['engagement_total'])
    spearman_r, spearman_p = spearmanr(df[feature], df['engagement_total'])
    print(f"{feature:30s} | Pearson: {pearson_r:7.4f} (p={pearson_p:.4f}) | Spearman: {spearman_r:7.4f} (p={spearman_p:.4f})")

## 7. Color Category Analysis - Original vs RGB Classification

In [ ]:
# Engagement by original color category
print("\n=== ENGAGEMENT BY ORIGINAL COLOR CATEGORY ===")
color_engagement = df.groupby('color_category').agg({
    'likes': ['mean', 'median', 'std', 'count'],
    'engagement_total': ['mean', 'median', 'std']
}).round(2)
print(color_engagement)

In [ ]:
# Engagement by RGB category
print("\n=== ENGAGEMENT BY RGB CATEGORY ===")
rgb_engagement = df.groupby('rgb_category').agg({
    'likes': ['mean', 'median', 'std', 'count'],
    'engagement_total': ['mean', 'median', 'std']
}).round(2)
print(rgb_engagement)

In [ ]:
# Box plots - Original Color Category
fig, axes = plt.subplots(1, 2, figsize=(18, 6))

df.boxplot(column='likes', by='color_category', ax=axes[0])
axes[0].set_title('Likes by Original Color Category')
axes[0].set_xlabel('Color Category')
axes[0].set_ylabel('Likes')
axes[0].tick_params(axis='x', rotation=45)

df.boxplot(column='engagement_total', by='color_category', ax=axes[1])
axes[1].set_title('Total Engagement by Original Color Category')
axes[1].set_xlabel('Color Category')
axes[1].set_ylabel('Total Engagement')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Box plots - RGB Category
fig, axes = plt.subplots(1, 2, figsize=(18, 6))

df.boxplot(column='likes', by='rgb_category', ax=axes[0])
axes[0].set_title('Likes by RGB Category')
axes[0].set_xlabel('RGB Category')
axes[0].set_ylabel('Likes')
axes[0].tick_params(axis='x', rotation=45)

df.boxplot(column='engagement_total', by='rgb_category', ax=axes[1])
axes[1].set_title('Total Engagement by RGB Category')
axes[1].set_xlabel('RGB Category')
axes[1].set_ylabel('Total Engagement')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 8. Statistical Tests - ANOVA and Kruskal-Wallis

In [ ]:
# ANOVA - Test if color categories have significantly different engagement
print("\n=== ANOVA TESTS ===")

# For original color categories
color_groups_likes = [group['likes'].values for name, group in df.groupby('color_category')]
f_stat_color, p_value_color = f_oneway(*color_groups_likes)
print(f"\nOriginal Color Category - Likes:")
print(f"  F-statistic: {f_stat_color:.4f}")
print(f"  p-value: {p_value_color:.4f}")
print(f"  Significant: {'YES' if p_value_color < 0.05 else 'NO'}")

# For RGB categories
rgb_groups_likes = [group['likes'].values for name, group in df.groupby('rgb_category')]
f_stat_rgb, p_value_rgb = f_oneway(*rgb_groups_likes)
print(f"\nRGB Category - Likes:")
print(f"  F-statistic: {f_stat_rgb:.4f}")
print(f"  p-value: {p_value_rgb:.4f}")
print(f"  Significant: {'YES' if p_value_rgb < 0.05 else 'NO'}")

In [ ]:
# Kruskal-Wallis (non-parametric alternative to ANOVA)
print("\n=== KRUSKAL-WALLIS TESTS ===")

# For original color categories
h_stat_color, p_value_kw_color = kruskal(*color_groups_likes)
print(f"\nOriginal Color Category - Likes:")
print(f"  H-statistic: {h_stat_color:.4f}")
print(f"  p-value: {p_value_kw_color:.4f}")
print(f"  Significant: {'YES' if p_value_kw_color < 0.05 else 'NO'}")

# For RGB categories
h_stat_rgb, p_value_kw_rgb = kruskal(*rgb_groups_likes)
print(f"\nRGB Category - Likes:")
print(f"  H-statistic: {h_stat_rgb:.4f}")
print(f"  p-value: {p_value_kw_rgb:.4f}")
print(f"  Significant: {'YES' if p_value_kw_rgb < 0.05 else 'NO'}")

## 9. Multiple Regression Analysis

In [ ]:
# Multiple regression to predict engagement
from sklearn.metrics import r2_score, mean_squared_error

# Select features for regression
feature_cols = ['mean_luminosity', 'std_luminosity', 'mean_saturation', 'std_saturation',
                'predominant_color_percentage', 'red', 'green', 'blue']

X = df[feature_cols]
y = df['likes']

# Fit model
model = LinearRegression()
model.fit(X, y)

# Predictions
y_pred = model.predict(X)

# Model performance
r2 = r2_score(y, y_pred)
rmse = np.sqrt(mean_squared_error(y, y_pred))

print("\n=== MULTIPLE REGRESSION RESULTS (Predicting Likes) ===")
print(f"R-squared: {r2:.4f}")
print(f"RMSE: {rmse:.2f}")
print(f"\nFeature Coefficients:")
for feature, coef in zip(feature_cols, model.coef_):
    print(f"  {feature:30s}: {coef:10.4f}")
print(f"  Intercept: {model.intercept_:.4f}")

In [ ]:
# Check for multicollinearity using VIF
print("\n=== VARIANCE INFLATION FACTOR (VIF) ===")
print("VIF > 10 indicates high multicollinearity\n")

vif_data = pd.DataFrame()
vif_data["Feature"] = feature_cols
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(feature_cols))]
print(vif_data)

## 10. Chi-Square Test for Categorical Variables

In [ ]:
# Create engagement level categories
df['engagement_level'] = pd.cut(df['likes'], 
                                  bins=[0, 20000, 50000, 100000, float('inf')],
                                  labels=['Low', 'Medium', 'High', 'Very High'])

# Chi-square test: Original color category vs engagement level
contingency_table_color = pd.crosstab(df['color_category'], df['engagement_level'])
chi2_color, p_color, dof_color, expected_color = chi2_contingency(contingency_table_color)

print("\n=== CHI-SQUARE TEST: Original Color Category vs Engagement Level ===")
print(f"Chi-square statistic: {chi2_color:.4f}")
print(f"p-value: {p_color:.4f}")
print(f"Degrees of freedom: {dof_color}")
print(f"Significant: {'YES' if p_color < 0.05 else 'NO'}")
print("\nContingency Table:")
print(contingency_table_color)

In [ ]:
# Chi-square test: RGB category vs engagement level
contingency_table_rgb = pd.crosstab(df['rgb_category'], df['engagement_level'])
chi2_rgb, p_rgb, dof_rgb, expected_rgb = chi2_contingency(contingency_table_rgb)

print("\n=== CHI-SQUARE TEST: RGB Category vs Engagement Level ===")
print(f"Chi-square statistic: {chi2_rgb:.4f}")
print(f"p-value: {p_rgb:.4f}")
print(f"Degrees of freedom: {dof_rgb}")
print(f"Significant: {'YES' if p_rgb < 0.05 else 'NO'}")
print("\nContingency Table:")
print(contingency_table_rgb)

## 11. Scatter Plots with Trend Lines

In [ ]:
# Scatter plots for key relationships
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Plot 1: std_luminosity vs likes
axes[0, 0].scatter(df['std_luminosity'], df['likes'], alpha=0.5)
axes[0, 0].set_xlabel('Std Luminosity')
axes[0, 0].set_ylabel('Likes')
axes[0, 0].set_title('Std Luminosity vs Likes')
z = np.polyfit(df['std_luminosity'], df['likes'], 1)
p = np.poly1d(z)
axes[0, 0].plot(df['std_luminosity'], p(df['std_luminosity']), "r--", alpha=0.8)

# Plot 2: predominant_color_percentage vs likes
axes[0, 1].scatter(df['predominant_color_percentage'], df['likes'], alpha=0.5, color='green')
axes[0, 1].set_xlabel('Predominant Color %')
axes[0, 1].set_ylabel('Likes')
axes[0, 1].set_title('Predominant Color % vs Likes')
z = np.polyfit(df['predominant_color_percentage'], df['likes'], 1)
p = np.poly1d(z)
axes[0, 1].plot(df['predominant_color_percentage'], p(df['predominant_color_percentage']), "r--", alpha=0.8)

# Plot 3: mean_saturation vs likes
axes[0, 2].scatter(df['mean_saturation'], df['likes'], alpha=0.5, color='orange')
axes[0, 2].set_xlabel('Mean Saturation')
axes[0, 2].set_ylabel('Likes')
axes[0, 2].set_title('Mean Saturation vs Likes')
z = np.polyfit(df['mean_saturation'], df['likes'], 1)
p = np.poly1d(z)
axes[0, 2].plot(df['mean_saturation'], p(df['mean_saturation']), "r--", alpha=0.8)

# Plot 4: red vs likes
axes[1, 0].scatter(df['red'], df['likes'], alpha=0.5, color='red')
axes[1, 0].set_xlabel('Red Channel')
axes[1, 0].set_ylabel('Likes')
axes[1, 0].set_title('Red Channel vs Likes')
z = np.polyfit(df['red'], df['likes'], 1)
p = np.poly1d(z)
axes[1, 0].plot(df['red'], p(df['red']), "k--", alpha=0.8)

# Plot 5: green vs likes
axes[1, 1].scatter(df['green'], df['likes'], alpha=0.5, color='green')
axes[1, 1].set_xlabel('Green Channel')
axes[1, 1].set_ylabel('Likes')
axes[1, 1].set_title('Green Channel vs Likes')
z = np.polyfit(df['green'], df['likes'], 1)
p = np.poly1d(z)
axes[1, 1].plot(df['green'], p(df['green']), "k--", alpha=0.8)

# Plot 6: blue vs likes
axes[1, 2].scatter(df['blue'], df['likes'], alpha=0.5, color='blue')
axes[1, 2].set_xlabel('Blue Channel')
axes[1, 2].set_ylabel('Likes')
axes[1, 2].set_title('Blue Channel vs Likes')
z = np.polyfit(df['blue'], df['likes'], 1)
p = np.poly1d(z)
axes[1, 2].plot(df['blue'], p(df['blue']), "k--", alpha=0.8)

plt.tight_layout()
plt.show()

## 12. Advanced Color Analysis - RGB vs Original Categories

In [ ]:
# Comparison of variance in engagement across classification schemes
print("\n=== VARIANCE COMPARISON ===")

# Original color categories
color_variance = df.groupby('color_category')['likes'].var()
print(f"\nOriginal Color Categories - Variance in Likes:")
print(color_variance)
print(f"Mean Variance: {color_variance.mean():.2f}")

# RGB categories
rgb_variance = df.groupby('rgb_category')['likes'].var()
print(f"\nRGB Categories - Variance in Likes:")
print(rgb_variance)
print(f"Mean Variance: {rgb_variance.mean():.2f}")

In [ ]:
# Create a comparison visualization
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Original categories
color_means = df.groupby('color_category')['likes'].mean().sort_values(ascending=False)
axes[0].barh(range(len(color_means)), color_means.values)
axes[0].set_yticks(range(len(color_means)))
axes[0].set_yticklabels(color_means.index)
axes[0].set_xlabel('Average Likes')
axes[0].set_title('Average Likes by Original Color Category')
axes[0].grid(axis='x', alpha=0.3)

# RGB categories
rgb_means = df.groupby('rgb_category')['likes'].mean().sort_values(ascending=False)
axes[1].barh(range(len(rgb_means)), rgb_means.values, color='steelblue')
axes[1].set_yticks(range(len(rgb_means)))
axes[1].set_yticklabels(rgb_means.index)
axes[1].set_xlabel('Average Likes')
axes[1].set_title('Average Likes by RGB Category')
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

## 13. Key Insights and Statistical Inferences

In [ ]:
print("\n" + "="*80)
print("KEY INSIGHTS AND STATISTICAL INFERENCES")
print("="*80)

# 1. Overall correlation strength
print("\n1. CORRELATION ANALYSIS:")
engagement_corrs = pearson_corr['likes'].drop(['likes', 'comments', 'engagement_total']).abs().sort_values(ascending=False)
print(f"   - Strongest correlations with likes: {engagement_corrs.head(3).to_dict()}")
print(f"   - Overall: Correlations are weak (all < 0.2), indicating engagement is multifactorial")

# 2. Color category impact
print("\n2. COLOR CATEGORY IMPACT:")
print(f"   - Original Categories ANOVA p-value: {p_value_color:.4f}")
print(f"   - RGB Categories ANOVA p-value: {p_value_rgb:.4f}")
if p_value_color < 0.05:
    print("   - Original color categories show SIGNIFICANT differences in engagement")
else:
    print("   - Original color categories show NO significant differences in engagement")
if p_value_rgb < 0.05:
    print("   - RGB categories show SIGNIFICANT differences in engagement")
else:
    print("   - RGB categories show NO significant differences in engagement")

# 3. Top performing categories
print("\n3. TOP PERFORMING CATEGORIES:")
print(f"   Original Classification:")
print(f"   - Best: {color_means.index[0]} (avg: {color_means.values[0]:.0f} likes)")
print(f"   - Worst: {color_means.index[-1]} (avg: {color_means.values[-1]:.0f} likes)")
print(f"   RGB Classification:")
print(f"   - Best: {rgb_means.index[0]} (avg: {rgb_means.values[0]:.0f} likes)")
print(f"   - Worst: {rgb_means.index[-1]} (avg: {rgb_means.values[-1]:.0f} likes)")

# 4. Model performance
print("\n4. PREDICTIVE MODEL:")
print(f"   - R-squared: {r2:.4f}")
print(f"   - This means {r2*100:.2f}% of variance in likes is explained by the visual features")
print(f"   - Remaining {(1-r2)*100:.2f}% is due to other factors (content, timing, audience, etc.)")

# 5. RGB insights
print("\n5. RGB CHANNEL INSIGHTS:")
print(f"   - Red channel correlation with likes: {pearson_corr.loc['red', 'likes']:.4f}")
print(f"   - Green channel correlation with likes: {pearson_corr.loc['green', 'likes']:.4f}")
print(f"   - Blue channel correlation with likes: {pearson_corr.loc['blue', 'likes']:.4f}")

# 6. Comparison of classification schemes
print("\n6. CLASSIFICATION SCHEME COMPARISON:")
print(f"   - Original scheme has {df['color_category'].nunique()} categories")
print(f"   - RGB scheme has {df['rgb_category'].nunique()} categories")
print(f"   - RGB scheme provides {'more' if df['rgb_category'].nunique() > df['color_category'].nunique() else 'less'} granular classification")

print("\n" + "="*80)

## 14. Summary and Conclusions

In [ ]:
print("\n" + "="*80)
print("SUMMARY AND CONCLUSIONS")
print("="*80)
print("""
This comprehensive analysis of the full Instagram fashion image dataset reveals:

1. CORRELATION PATTERNS:
   - Visual features show weak correlations with engagement (typically < 0.2)
   - Std luminosity and predominant color percentage are among the strongest predictors
   - RGB channels individually show minimal correlation with engagement

2. COLOR ANALYSIS:
   - Both classification schemes (original 8-9 categories and RGB-based) show patterns
   - Statistical tests reveal whether color significantly impacts engagement
   - RGB classification provides an alternative perspective on color impact

3. STATISTICAL SIGNIFICANCE:
   - ANOVA and Kruskal-Wallis tests determine if color categories matter
   - Chi-square tests reveal associations between color and engagement levels
   - Multiple regression quantifies the combined effect of visual features

4. PRACTICAL IMPLICATIONS:
   - Visual features explain a limited portion of engagement variance
   - Content quality, timing, audience, and other factors are crucial
   - Color choices may have modest but measurable effects

5. RGB vs ORIGINAL CLASSIFICATION:
   - RGB classification offers a more fundamental color perspective
   - Original classification captures semantic color meanings
   - Both have value depending on the analysis goal
   - Findings differ from small dataset analysis, showing importance of large samples
""")
print("="*80)